In [2]:
from keras import utils   

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Dropout
#from keras.utils import np_utils

In [4]:
path = r'C:\Users\jinu5\Desktop\careerfoundry\ML\DataSet\original data'
df = pd.read_csv(os.path.join(path, 'Dataset-weather-prediction-dataset-processed_located.csv'), index_col=False)
df_y = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col=False)

In [5]:
tf.random.set_seed(42)

### drop DATE and MONTH`

In [6]:
dfx = df.drop(['DATE', 'MONTH'], axis=1)
dfy = df_y.drop(['DATE'], axis=1)

### Two types of observations are missing multiple years for most weather stations. Remove them.

In [7]:
dfx = dfx.drop(columns=['Unnamed: 0'])

In [8]:
dfx

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.0180,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.0180,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.0180,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.0180,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.0180,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,2.1,0.79,1.0248,1.34,0.22,0,7.7,15.9,11.4,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22946,6,2.1,0.77,1.0244,1.34,0.22,0,5.4,16.7,14.3,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22947,4,2.1,0.76,1.0227,1.34,0.22,0,6.1,16.7,13.1,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22948,5,2.1,0.80,1.0212,1.34,0.22,0,5.8,15.4,11.6,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5


In [9]:
var_list=dfx.columns.tolist()

In [10]:
var_list

['BASEL_cloud_cover',
 'BASEL_wind_speed',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_snow_depth',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_wind_speed',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_wind_speed',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_

### There are three individual observations that need to be filled in. Assume nearby stations have similar weather, then pick one to copy the data from (Ljubljana is near Kassel, Sonnblick is near Munchen, and Olso is close enough to Stockholm).

To do this, I reviewed the dataset in Excel. and I found that some cities like Kasel don't have some vars. (i.g. Kasel does not have could_cover)  
- KASSEL_could_cover could be substituted with LJUBLJANA_cloud_cover.
- almost all records in Kassel from 2013-01-02 are the same, which means that all records in Kassel from that time are not significant. Hence, these would be better to be substituted with Ljubljana records.
- I cleaned these data in Excel and saved df_cleaned.csv

### In case of Valentia, like Kassel, Almost all of the data from 2016 through 2022 is duplicated. 

### cleaning records.

1. creating Kassel's cloud cover column with Ljubljana cloud cover (copy)
2. the first 366 barometric pressure records for Ljubljana are duplicates, so we replace them with Kassel's data
3. from row 19666 to the last row, all data from Kassel are duplicates, so we replace them with records from Ljubljana
4. no barometric pressure data for Munich, so we replaced it with a record from Sonnblick
5. duplicate MUNCHEN_global_radiation rows 22038 through 22251, replace with Sonnblick, duplicate MUNCHEN_global_radiation rows 22283 through 22494
6. duplicate records SONNBLICK_global_radiation SONNBLICK_precipitation SONNBLICK_sunshine from 22768 to end, replace with Munich records
7.No humidity data for Stockholm. Replaced with Oslo
8. Stockholm first 366 barometric pressure duplicates
9. duplicate OSLO_sunshine 18599 through 19336
10. duplicate STOCKHOLM_pressure from 18995 to 19327 
11. duplicate STOCKHOLM_cloud_cover from 18995 to the end
12. OSLO_global_radiation OSLO_precipitation STOCKHOLM_global_radiation STOCKHOLM_precipitation
Last 31 duplicate records 

13. for Valencia, duplicates from 18415 to last in all columns except VALENTIA_global_radiation VALENTIA_temp_max

In [11]:
#re-import cleaned dataset 
df = pd.read_csv(os.path.join(path, 'df_cleaned.csv'), index_col=False)

In [12]:
df

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


### Ensure the layers can be fed to the deep learning model correctly. You’ll need to split the observations (X) into 15 groups of 9 types of observations, and your labels (y) should also be in 15 groups (it doesn’t need to be transformed or reshaped). The final shapes should be X = (22950, 15, 9) and y = (22950, 15).
When reshaping a 3-D object, you can use the following code X = X.reshape(-1,15,9), where -1 means “the shape that fits with the rest.”

In [13]:
city=[]
obtype=[]
for col in var_list:
    a,b = col.split('_', 1)
    if a not in city:
        city.append(a)
    if b not in obtype:
        obtype.append(b)
    

In [14]:
city

['BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA']

In [15]:
city_dict = dict(enumerate(city))

In [16]:
city_dict

{0: 'BASEL',
 1: 'BELGRADE',
 2: 'BUDAPEST',
 3: 'DEBILT',
 4: 'DUSSELDORF',
 5: 'HEATHROW',
 6: 'KASSEL',
 7: 'LJUBLJANA',
 8: 'MAASTRICHT',
 9: 'MADRID',
 10: 'MUNCHENB',
 11: 'OSLO',
 12: 'SONNBLICK',
 13: 'STOCKHOLM',
 14: 'VALENTIA'}

In [17]:
obtype

['cloud_cover',
 'wind_speed',
 'humidity',
 'pressure',
 'global_radiation',
 'precipitation',
 'snow_depth',
 'sunshine',
 'temp_mean',
 'temp_min',
 'temp_max']

In [18]:
X=df.to_numpy()
y=dfy.to_numpy()

In [19]:
X = X.reshape(-1,15,9)

In [20]:
X.shape

(22950, 15, 9)

In [21]:
y.shape

(22950, 15)

### Scale X

In [23]:
from sklearn.preprocessing import StandardScaler

#Scaling the data
scaler = StandardScaler()

#Fit the train data
scaler.fit(X)

#Scaling the X data
X = scaler.transform(X)
#X_test_sc = scaler.transform(X_test)

print(X[:10])

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [34]:
#ValueError: Found array with dim 3. StandardScaler expected <= 2.
# X is 3 dim now, due to this, scaling makes an error. 
# 1. re-import cleaned dataset 
df = pd.read_csv(os.path.join(path, 'df_cleaned.csv'), index_col=False)
# 2. create X again
X=df.to_numpy()
# 3. scaling
#Fit the train data
scaler.fit(X)

#Scaling the X data
X = scaler.transform(X)

# 4. reshape X
X = X.reshape(-1,15,9)

In [37]:
print(X[:3])

[[[ 6.60513663e-01  8.26096599e-01 -1.94863388e-03 -1.10106604e+00
   -2.65147724e-01 -9.02918175e-01 -5.28623012e-01 -8.45651922e-01
   -4.78356271e-01]
  [-1.20643263e+00  9.05270489e-01  3.21770762e-01 -7.43148516e-01
   -3.71315465e-01  2.83982584e-01 -1.01687613e+00 -1.22021042e+00
   -9.49202784e-01]
  [-2.55240242e-01 -1.00976762e-02 -5.87602906e-03 -1.11853703e+00
   -3.21867660e-01 -7.73377865e-01 -1.09916317e+00 -1.11943125e+00
   -1.13683915e+00]
  [ 7.13781743e-01  3.57838582e-01 -1.23247143e+00 -1.19701310e+00
    4.96188574e-02 -1.07731152e+00 -1.14355781e-01  2.63326027e-01
   -4.11047464e-01]
  [ 1.23252642e+00  6.25745610e-01 -5.60634075e-03 -1.15757474e+00
   -3.05937674e-01 -1.01490758e+00 -1.05836016e-01  6.05549863e-02
   -4.10422790e-01]
  [ 8.33387413e-01  1.33257233e+00 -1.37659230e+00 -1.18737020e+00
    1.41032181e-01 -1.06472113e+00 -1.06468934e-01  3.93469809e-01
   -1.03849206e+00]
  [ 1.20549184e+00  4.40698493e-01 -7.87442513e-01 -1.00839885e+00
    4.954

In [25]:
X.shape

(22950, 15, 9)

### Split your data

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(17212, 15, 9)
(5738, 15, 9)
(17212, 15)
(5738, 15)


### Starting from either the CNN or RNN sample script for the HAR data, create a Keras layered model.
### In the case of Scaling data, I will run my best model of unscaled directly

In [49]:
def _count_classes(y):
    return len(set([tuple(category.tolist()) for category in y]))
y_train_integer = np.argmax(y_train, axis=1)
y_test_integer = np.argmax(y_test, axis=1)

In [46]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([city_dict[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([city_dict[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [47]:
epochs = 60  
batch_size = 32  
n_hidden = 64  
  
timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])  
n_classes = _count_classes(y_train)  
  
model = Sequential()  
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))  
model.add(Dense(16, activation='relu'))  
model.add(MaxPooling1D())  
model.add(Flatten())  
model.add(Dense(n_classes, activation='softmax')) #sigmoid  
  
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
model.fit(X_train, y_train_integer, batch_size=batch_size, epochs=epochs, verbose=0)  
  #Evaluate  
print(confusion_matrix(y_test, model.predict(X_test)))  

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3488        97         7       4           4         5       0   
BELGRADE       35      1022        16       2           0         1       0   
BUDAPEST       11        39       134       4           0         2       3   
DEBILT          5         6         3      58           3         4       0   
DUSSELDORF      4         3         3       4           9         5       0   
HEATHROW        6         3         2       2           0        50       1   
KASSEL          0         2         1       0           0         1       5   
LJUBLJANA       5         1         2       1           0         3       0   
MAASTRICHT      5         0         0       0           1         1       1   
MADRID         27         8         7       2           2         3       1   
MUNCHENB   

In [50]:
loss, accuracy = model.evaluate(X_test, y_test_integer, verbose=2)
print("Test Accuracy:", accuracy)

180/180 - 1s - 7ms/step - accuracy: 0.9062 - loss: 0.2828
Test Accuracy: 0.9062390923500061


# Result
The accuary of sacled data with same model decreased from 0.94 in unscaled to 0.90
It seems that Scaling/unscaling yields different results from the same model, and scaling data does not always increase accuracy. 